In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-03-09 20:01:48,653 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-09 20:01:48,656 - settings - Setting database.user to celiib
INFO - 2021-03-09 20:01:48,658 - settings - Setting database.password to newceliipass
INFO - 2021-03-09 20:01:48,669 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-03-09 20:01:48,670 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-09 20:01:48,686 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-03-09 20:01:49,107 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-09 20:01:49,110 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-09 20:01:49,112 - settings - Setting database.user to celiib
INFO - 2021-03-09 20:01:49,113 - settings - Setting database.password to newceliipass
INFO - 2021-03-09 20:01:49,114 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-09 20:01:49,115 - settings - Setting database.user to celiib
INFO - 2021-03-09 20:01:49,116 - settings - Setting database.password to newceliipass
INFO - 2021-03-09 20:01:49,119 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-09 20:01:49,556 - settings - Setting enable_python_native_blobs to True


In [3]:
#so that it will have the adapter defined
from datajoint_utils import *

In [4]:
test_mode = True

# Debugging the contains method

In [5]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-03-09 20:01:54,299 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-09 20:01:54,300 - settings - Setting database.user to celiib
INFO - 2021-03-09 20:01:54,301 - settings - Setting database.password to newceliipass
INFO - 2021-03-09 20:01:54,303 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-09 20:01:54,304 - settings - Setting database.user to celiib
INFO - 2021-03-09 20:01:54,304 - settings - Setting database.password to newceliipass
INFO - 2021-03-09 20:01:54,310 - settings - Setting enable_python_native_blobs to True


Sleeping 71 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-03-09 20:01:54,792 - settings - Setting enable_python_native_blobs to True


In [6]:
#(schema.jobs & "table_name='__decomposition'").delete()

# Defining the Table

In [7]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [8]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 6450


In [17]:
decimation_version = 0
decimation_ratio = 0.25

current_allen_table = (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source = ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")  & 
              current_allen_table)
key_source                 

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134988608634,0,0.25
864691135065024068,0,0.25
864691135155711844,0,0.25
864691135210232512,0,0.25
864691135341057989,0,0.25
864691135345291423,0,0.25
864691135373423816,0,0.25
864691135373676873,0,0.25
864691135382990571,0,0.25
864691135463533125,0,0.25


In [ ]:
#minnie.DecompositionVersions.insert1(dict(process_version=6,description="3/4: retry limb processing with meshparty if meshafterparty fails"),skip_duplicates=True)

In [13]:
# schema.external['decomposition'].delete(delete_external_files=True)
# schema.external['somas'].delete(delete_external_files=True)
# schema.external['faces'].delete(delete_external_files=True)
# # schema.external['decimated_meshes'].delete(delete_external_files=True)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


[]

In [ ]:
# key_source = ((minnie.Decimation).proj(decimation_version='version') & 
#                             "decimation_version=" + str(decimation_version) &
#                        f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")  & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj())               
# minnie.Decomposition() & key_source

In [12]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25
process_version = 6

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    process_version : int unsigned #the version of the preprocessing pipeline run
    index : tinyint unsigned  #the index of the neuron object that resulted from that mesh (indexed starting at 0)
    ---
    multiplicity=null    : tinyint unsigned             # the number of somas found for this base segment
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """


    
    key_source = ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")  & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj())               
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)
        

        #3) Get the somas info 
        somas = du.get_soma_mesh_list(segment_id) 
        soma_ver = du.get_soma_mesh_list_ver(segment_id)
                  
        
        print(f"somas = {somas}")
                  
        #3b) Get the glia and nuclei information 
        glia_faces,nuclei_faces = du.get_segment_glia_nuclei_faces(segment_id,return_empty_list=True)
        
                  
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"],
        glia_faces=glia_faces,
        nuclei_faces = nuclei_faces,

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")

        #7) Pass stats and file location to insert
        new_key = dict(key,
                       ver = soma_ver,
                       process_version = process_version,
                       index = 0,
                       multiplicity=1,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)
        
        keys_to_delete = ["axon_length",
        "axon_area",
        "max_soma_volume",
        "max_soma_n_faces"]
        
        for k_to_delete in keys_to_delete:
            del new_key[k_to_delete]

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [11]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table)#.delete()
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,b0b22dd34002f2b5689041840713cc73,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,134501,216198,2021-03-09 13:22:19


In [ ]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

INFO - 2021-03-09 19:22:19,340 - autopopulate - Found 2 keys to populate
INFO - 2021-03-09 19:22:19,351 - connection - Transaction started
INFO - 2021-03-09 19:22:19,354 - autopopulate - Populating: {'segment_id': 864691136008578990, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started



---- Working on Neuron 864691136008578990 ----
somas = [[<trimesh.Trimesh(vertices.shape=(15794, 3), faces.shape=(31000, 3))>, <trimesh.Trimesh(vertices.shape=(15430, 3), faces.shape=(30522, 3))>], array([610.321, 610.321]), array([0.85, 0.85])]

--- Beginning preprocessing of 864691136008578990---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(15794, 3), faces.shape=(31000, 3))>, <trimesh.Trimesh(vertices.shape=(15430, 3), faces.shape=(30522, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(15794, 3), faces.shape=(31000, 3))>, <trimesh.Trimesh(vertices.shape=(15430, 3), faces.shape=(30522, 3))>]
soma_mesh_list_centers = [array([557811.46009877, 676656.54779663, 717464.9974294 ]), array([632953.95648089, 685749.53403111, 727124.35926119])]
Getting Glia and Nuclei Pieces Subtr

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/142301 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.80s/it]


...

segment_name before cgal = L0_17


  0%|          | 0/25 [00:00<?, ?it/s]

segment_name before cgal = L0_18


  0%|          | 0/43 [00:00<?, ?it/s]

segment_name before cgal = L0_19
